In [1]:
import cv2
import numpy as np
from inference_sdk import InferenceHTTPClient
import os
import tempfile

In [2]:
# Initialize the Roboflow inference client
CLIENT = InferenceHTTPClient(
    api_url="https://outline.roboflow.com",
    api_key="n7ScRPPA2ZMJoTCdUrBp"
)

MODEL_ID = "data_tonghop/2"

In [3]:
# Folder paths
# folder_1 = "C:\\Users\\USER\\Downloads"
# folder_2 = "\\Thesis\\Image Processing\\Jan 14 2025 - Batch 1"
input_folder = ".\\April_05_2025-Batch3"
done_folder = os.path.join(input_folder, "Done")
output_folder = os.path.join(input_folder, "outputs")

# Create folders if they don't exist
os.makedirs(done_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Padding for bounding boxes
padding = 10

In [4]:
# Process each video in the folder
for video_file in os.listdir(input_folder):
    if not video_file.endswith(".mp4"):
        continue

    video_path = os.path.join(input_folder, video_file)
    video_name = os.path.splitext(video_file)[0]

    # Create output subfolder for this video
    video_output_folder = os.path.join(output_folder, video_name)
    os.makedirs(video_output_folder, exist_ok=True)

    # Open the video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Output video path
    output_video_path = os.path.join(video_output_folder, f"{video_name}_processed.mp4")

    # Video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract depth and RGB sides
        depth_frame = frame[:, :frame_width // 2]
        rgb_frame = frame[:, frame_width // 2:]

        # Save the RGB frame to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_file:
            temp_image_path = temp_file.name
            cv2.imwrite(temp_image_path, rgb_frame)

        try:
            # Perform inference via Roboflow API
            response = CLIENT.infer(temp_image_path, model_id=MODEL_ID)

            # Counter for bounding boxes in this frame
            bbox_count = 1

            # Parse response for bounding boxes
            for prediction in response['predictions']:
                x = int(prediction['x'])
                y = int(prediction['y'])
                width = int(prediction['width'])
                height = int(prediction['height'])

                # Calculate bounding box coordinates (RGB side)
                x1 = max(x - width // 2 - padding, 0)
                y1 = max(y - height // 2 - padding, 0)
                x2 = min(x + width // 2 + padding, rgb_frame.shape[1] - 1)
                y2 = min(y + height // 2 + padding, rgb_frame.shape[0] - 1)

                # Draw the bounding box on RGB frame
                cv2.rectangle(rgb_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Map bounding box to depth frame
                depth_x1 = x1
                depth_y1 = y1
                depth_x2 = x2
                depth_y2 = y2

                # Crop the region from the depth frame
                cropped_rgb = rgb_frame[y1:y2, x1:x2]
                cropped_depth = depth_frame[depth_y1:depth_y2, depth_x1:depth_x2]

                # Save the cropped frames with incremental numbering
                cropped_file_path_rgb = os.path.join(video_output_folder, f"frame_{frame_count}_rgb_crop_{bbox_count}.png")
                cropped_file_path_depth = os.path.join(video_output_folder, f"frame_{frame_count}_depth_crop_{bbox_count}.png")
                cv2.imwrite(cropped_file_path_rgb, cropped_rgb)
                cv2.imwrite(cropped_file_path_depth, cropped_depth)

                # Increment bounding box counter
                bbox_count += 1

        finally:
            # Clean up the temporary file
            if os.path.exists(temp_image_path):
                os.remove(temp_image_path)

        # Combine the RGB frame with the depth side (left half)
        frame[:, frame_width // 2:] = rgb_frame

        # Write the frame with bounding boxes to the output video
        out.write(frame)

        frame_count += 1

    # Release resources for this video
    cap.release()
    out.release()

    # Move processed video to "Done" folder
    os.rename(video_path, os.path.join(done_folder, video_file))


HTTPCallErrorError: HTTPCallErrorError(description='403 Client Error: Forbidden for url: https://outline.roboflow.com/data_tonghop/2?api_key=n7***Bp&disable_active_learning=False', api_message='Forbidden',status_code=403)

In [ ]:
print(f"Processing complete. Outputs saved in {output_folder}")

Processing complete. Outputs saved in .\Jan_14_2025-Batch1\outputs
